In [ ]:
# split the training into 0.75 train and 0.25 test for cross validation
from sklearn.model_selection import StratifiedShuffleSplit,RepeatedStratifiedKFold
shuffle_split = StratifiedShuffleSplit(test_size=0.25,train_size=0.75,n_splits=35,random_state=87951)
#kfold = RepeatedStratifiedKFold(n_splits=25, n_repeats=10, random_state=87951)
param_grid = {
                'alpha'     : [0.1,1,10,0.01],
                'l1_ratio'  :  np.arange(0.40,1.00,0.10),
                'tol'       : [0.0001,0.001]
            }
eNet = ElasticNet(max_iter=10000)
grid_search = GridSearchCV(eNet, 
                           param_grid, 
                           scoring='roc_auc', 
                           cv = shuffle_split,
                           return_train_score=True,
                           n_jobs = -1)
grid_search.fit(train_df,target)

In [2]:
# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

from xgboost import XGBClassifier
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot
import shap
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc


random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data

M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = M_Y_train_ready.astype('int')

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)

F_Y_train_ready = F_Y_train_ready.astype('int')

M_X_test_ready = pickle.load(open("Test_samples\M_X_test_ready.pkl", 'rb'))
M_Y_test_ready = pickle.load(open("Test_samples\M_Y_test.pkl", 'rb'))
M_X_test_ready = pd.concat(M_X_test_ready.values(), join='inner', ignore_index=True)
M_Y_test_ready = pd.concat(M_Y_test_ready.values(), join='inner', ignore_index=True)

F_X_test_ready = pickle.load(open("Test_samples\F_X_test_ready.pkl", 'rb'))
F_Y_test_ready = pickle.load(open("Test_samples\F_Y_test.pkl", 'rb'))
F_X_test_ready = pd.concat(F_X_test_ready.values(), join='inner', ignore_index=True)
F_Y_test_ready = pd.concat(F_Y_test_ready.values(), join='inner', ignore_index=True)

CM_test_ready = pickle.load(open("Left_one_out_samples\CM_test_ready.pkl", 'rb'))
CM_Y_test = pickle.load(open("Left_one_out_samples\CM_Y_test.pkl", 'rb'))
CM_test_ready = pd.concat(CM_test_ready.values(), join='inner', ignore_index=True)
CM_Y_test = pd.concat(CM_Y_test.values(), join='inner', ignore_index=True)


CF_test_ready = pickle.load(open("Left_one_out_samples\CF_test_ready.pkl", 'rb'))
CF_Y_test = pickle.load(open("Left_one_out_samples\CF_Y_test.pkl", 'rb'))
CF_test_ready = pd.concat(CF_test_ready.values(), join='inner', ignore_index=True)
CF_Y_test = pd.concat(CF_Y_test.values(), join='inner', ignore_index=True)


M_selected_features = ['Age', 'Relationship with family head','Respondent live in household',
                        'Known HIV status','Bought/sold sex in the past 12 months',
                        'Condom was used at last paid sex in past 12 months','LAg: recent/long term infection',
                        'CD4 category','on ART','ART initiated past 12 months those ever on ART',
                        'last sex partner relations past 12 months','Sick to work last three months',
                        'Ever attended school','Ever enrolled in school','Highest level of education',
                        'Highest grade at that school level', 'Work for payment in last 12 months',
                        'Ever married/lived together','Avoiding pregnancy','CFcumcision status',
                        'Age at fFst sex', 'Ever sought TB treatment','Alcohol drink frequency',
                        'Urban area indicator','Wealth quintile', 'Sexual intercourse in the past 12 months',
                        'Ever had sexual intercourse'] 
M_X_train_ready_selected = M_X_train_ready[M_selected_features]
F_selected_features = ['Age', 'Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married/lived together',
       'Number of pregnancies',
       'Ever had a pregnancy that resulted in a live bFth',
       'No. of childen given bFth since 2012', 'Pregnacy status now',
       'Avoiding pregnancy', 'Age at fFst sex', 'Ever sought TB treatment',
       'Alcohol drink frequency', 'Urban area indicator', 'Known HIV status',
       'Wealth quintile', 'tested HIV at pregnancy',
       'Delivered 12 months preceeding survey',
       'Self-reported ARV status during pregnancy', 'Breastfed last child',
       'Sexual intercourse past 12 months', 'Ever had sexual intercourse',
       'Bought/sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent/long term infection', 'CD4 category', 'On ART',
       'ART initiated past 12 months those ever on ART',
       'Whether ARVs detected']

F_selected_features = []
F_X_train_ready_selected = F_X_train_ready[F_selected_features]
M_X_test_ready_selected = M_X_test_ready[M_selected_features]
F_X_test_ready_selected = F_X_test_ready[F_selected_features]
CM_test_ready_selected = CM_test_ready[M_selected_features]
CF_test_ready_selected = CF_test_ready[F_selected_features]

#================================================================================================================================
# parameters space, models definition and random grid search
#================================================================================================================================

# stratified k-fold and model initialization
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)
scoring = ['f1', 'recall', 'precision']
# scoring = f1_score.make_scorer

# param_grid = {
#                 'alpha'     : [0.1,1,10,0.01],
#                 'l1_ratio'  :  np.arange(0.40,1.00,0.10),
#                 'tol'       : [0.0001,0.001]
#             }

# from sklearn.linear_model import LogisticRegression
# grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
# logreg=LogisticRegression(max_iter=200)
# M_en_sample=GridSearchCV(logreg,grid,scoring = scoring,
#                              cv=kfold,
#                             n_jobs=-1,
                            
#                               verbose=2,
#                              refit='f1')
# F_en_sample=GridSearchCV(logreg,grid,scoring = scoring,
#                              cv=kfold,
#                             n_jobs=-1,
                            
#                               verbose=2,
#                              refit='f1')
# eNet = ElasticNet(max_iter=50)

# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV

# Setup the hyperparameter grid
# Larger c means less regularization. Smaller c means higher regularization.
# Regularization is an extra term added to the original loss function. Penalizes large value of coefficients.
# Hence training accuracy is decreased but improves test accuracy 
# since large coefficients that are overfitting the model is reduced. 

# c_space = np.logspace(-5, 8, 15) # 15 values between e~-5 to e^+8
# param_grid = {'C' : c_space,
#              'penalty' : ['l1', 'l2']}

# Create the classifier: logreg
# logreg = LogisticRegression()

# Instantiate the GridSearchCV object: logreg_cv
# logreg_cv = GridSearchCV(logreg, param_grid, scoring='accuracy', cv=5, n_jobs=-1)
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV

grid = {'C': np.logspace(-3,3,7), 'penalty': ['l1', 'l2']}
# C and penalty are logistic regression regularization parameters
# If C is too small model is underfitted, if C is too big model is overfitted.
# l1 and l2 are regularization loss functions (l1=lasso, l2=ridge)

# Creating the model:
lr = LogisticRegression(max_iter=250) 



# Creating GridSearchCV model:
M_en_sample = GridSearchCV(lr, grid, scoring,
                             cv=kfold,
                            n_jobs=-1,
#                             randomseed= 5,
                              verbose=2,
                             refit='f1')
F_en_sample = GridSearchCV(lr, grid, scoring,
                             cv=kfold,
                            n_jobs=-1,
#                             randomseed= 5,
                              verbose=2,
                             refit='f1')# Using lr model, grid parameters and cross validation of 10 (10 times of accuracy calculation will be applied) 

# Training the model:

# M_en_sample = GridSearchCV(eNet, 
#                            param_grid, 
#                           scoring='accuracy',
#                              cv=kfold,
#                             n_jobs=-1,
                            
#                               verbose=2,
#                              refit='f1')

# F_en_sample = GridSearchCV(eNet, 
#                            param_grid, 
#                            scoring='accuracy',
#                              cv=kfold,
#                             n_jobs=-1,
#                             random_state=random_seed,
#                               verbose=2,
#                              refit='f1')

M_en_sample.fit(M_X_train_ready_selected, M_Y_train_ready)
dump(M_en_sample, "en_sample\M_en_sample_data.joblib")

F_en_sample.fit(F_X_train_ready_selected, F_Y_train_ready)
dump(F_en_sample, "en_sample\F_en_sample_data.joblib")


Pass scoring=['f1', 'recall', 'precision'] as keyword args. From version 0.25 passing these as positional arguments will result in an error


Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   39.9s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   53.1s finished


Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    0.4s finished


ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

In [ ]:


# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
from matplotlib import pyplot
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve


random_seed = 5

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data

M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)

M_X_test_ready = pickle.load(open("Test_samples\M_X_test_ready.pkl", 'rb'))
M_Y_test_ready = pickle.load(open("Test_samples\M_Y_test.pkl", 'rb'))
M_X_test_ready = pd.concat(M_X_test_ready.values(), join='inner', ignore_index=True)
M_Y_test_ready = pd.concat(M_Y_test_ready.values(), join='inner', ignore_index=True)

F_X_test_ready = pickle.load(open("Test_samples\F_X_test_ready.pkl", 'rb'))
F_Y_test_ready = pickle.load(open("Test_samples\F_Y_test.pkl", 'rb'))
F_X_test_ready = pd.concat(F_X_test_ready.values(), join='inner', ignore_index=True)
F_Y_test_ready = pd.concat(F_Y_test_ready.values(), join='inner', ignore_index=True)

CM_test_ready = pickle.load(open("Left_one_out_samples\CM_test_ready.pkl", 'rb'))
CM_Y_test = pickle.load(open("Left_one_out_samples\CM_Y_test.pkl", 'rb'))
CM_test_ready = pd.concat(CM_test_ready.values(), join='inner', ignore_index=True)
CM_Y_test = pd.concat(CM_Y_test.values(), join='inner', ignore_index=True)


CF_test_ready = pickle.load(open("Left_one_out_samples\CF_test_ready.pkl", 'rb'))
CF_Y_test = pickle.load(open("Left_one_out_samples\CF_Y_test.pkl", 'rb'))
CF_test_ready = pd.concat(CF_test_ready.values(), join='inner', ignore_index=True)
CF_Y_test = pd.concat(CF_Y_test.values(), join='inner', ignore_index=True)


M_selected_features = ['Age', 'Relationship with family head','Respondent live in household',
                        'Known HIV status','Bought/sold sex in the past 12 months',
                        'Condom was used at last paid sex in past 12 months','LAg: recent/long term infection',
                        'CD4 category','on ART','ART initiated past 12 months those ever on ART',
                        'last sex partner relations past 12 months','Sick to work last three months',
                        'Ever attended school','Ever enrolled in school','Highest level of education',
                        'Highest grade at that school level', 'Work for payment in last 12 months',
                        'Ever married/lived together','Avoiding pregnancy','CFcumcision status',
                        'Age at fFst sex', 'Ever sought TB treatment','Alcohol drink frequency',
                        'Urban area indicator','Wealth quintile', 'Sexual intercourse in the past 12 months',
                        'Ever had sexual intercourse'] 
M_X_train_ready_selected = M_X_train_ready[M_selected_features]
F_selected_features = ['Age', 'Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married/lived together',
       'Number of pregnancies',
       'Ever had a pregnancy that resulted in a live bFth',
       'No. of childen given bFth since 2012', 'Pregnacy status now',
       'Avoiding pregnancy', 'Age at fFst sex', 'Ever sought TB treatment',
       'Alcohol drink frequency', 'Urban area indicator', 'Known HIV status',
       'Wealth quintile', 'tested HIV at pregnancy',
       'Delivered 12 months preceeding survey',
       'Self-reported ARV status during pregnancy', 'Breastfed last child',
       'Sexual intercourse past 12 months', 'Ever had sexual intercourse',
       'Bought/sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent/long term infection', 'CD4 category', 'On ART',
       'ART initiated past 12 months those ever on ART',
       'Whether ARVs detected']
    
     
F_X_train_ready_selected = F_X_train_ready[F_selected_features]
M_X_test_ready_selected = M_X_test_ready[M_selected_features]
F_X_test_ready_selected = F_X_test_ready[F_selected_features]
CM_test_ready_selected = CM_test_ready[M_selected_features]
CF_test_ready_selected = CF_test_ready[F_selected_features]


#================================================================================================================================
# parameters space, models definition and random grid search
#================================================================================================================================

# stratified k-fold and model initialization
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)
scoring = ['f1', 'recall', 'precision']


param_grid = {
                'alpha'     : [0.1,1,10,0.01],
                'l1_ratio'  :  np.arange(0.40,1.00,0.10),
                'tol'       : [0.0001,0.001]
            }
eNet = ElasticNet(max_iter=50)
M_grid_search = GridSearchCV(eNet, 
                           param_grid, 
                           scoring=scoring, 
                           cv = kfold,
                           n_jobs=-1,
                         verbose=2,
                         refit='f1')

F_grid_search = GridSearchCV(eNet, 
                           param_grid, 
                           scoring=scoring, 
                           cv = kfold,
                           n_jobs=-1,
                           verbose=2,
                         refit='f1')

M_grid_search.fit(M_X_train_ready_selected, M_Y_train_ready)
dump(M_grid_search, "M_grid_search\M_grid_search_sample_data.joblib")

F_grid_search.fit(F_X_train_ready_selected, F_Y_train_ready)
dump(F_grid_search, "F_grid_search\F_grid_search_sample_data.joblib")




In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
from matplotlib import pyplot
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc


from sklearn.metrics import r2_score

# import matplotlib.pyplot as plt
# %matplotlib inline


random_seed = 5

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data

M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)

CM_test_ready = pickle.load(open("Left_one_out_samples\CM_test_ready.pkl", 'rb'))
CM_Y_test_ready = pickle.load(open("Left_one_out_samples\CM_Y_test.pkl", 'rb'))
CM_test_ready = pd.concat(CM_test_ready.values(), join='inner', ignore_index=True)
CM_Y_test_ready = pd.concat(CM_Y_test_ready.values(), join='inner', ignore_index=True)


CF_test_ready = pickle.load(open("Left_one_out_samples\CF_test_ready.pkl", 'rb'))
CF_Y_test_ready = pickle.load(open("Left_one_out_samples\CF_Y_test.pkl", 'rb'))
CF_test_ready = pd.concat(CF_test_ready.values(), join='inner', ignore_index=True)
CF_Y_test_ready = pd.concat(CF_Y_test_ready.values(), join='inner', ignore_index=True)
M_selected_features = ['Age', 'Relationship with family head','Respondent live in household',
                        'Known HIV status','Bought/sold sex in the past 12 months',
                        'Condom was used at last paid sex in past 12 months','LAg: recent/long term infection',
                        'CD4 category','on ART','ART initiated past 12 months those ever on ART',
                        'last sex partner relations past 12 months','Sick to work last three months',
                        'Ever attended school','Ever enrolled in school','Highest level of education',
                        'Highest grade at that school level', 'Work for payment in last 12 months',
                        'Ever married/lived together','Avoiding pregnancy','CFcumcision status',
                        'Age at fFst sex', 'Ever sought TB treatment','Alcohol drink frequency',
                        'Urban area indicator','Wealth quintile', 'Sexual intercourse in the past 12 months',
                        'Ever had sexual intercourse'] 
M_X_train_ready_selected = M_X_train_ready[M_selected_features]
F_selected_features = ['Age', 'Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married/lived together',
       'Number of pregnancies',
       'Ever had a pregnancy that resulted in a live bFth',
       'No. of childen given bFth since 2012', 'Pregnacy status now',
       'Avoiding pregnancy', 'Age at fFst sex', 'Ever sought TB treatment',
       'Alcohol drink frequency', 'Urban area indicator', 'Known HIV status',
       'Wealth quintile', 'tested HIV at pregnancy',
       'Delivered 12 months preceeding survey',
       'Self-reported ARV status during pregnancy', 'Breastfed last child',
       'Sexual intercourse past 12 months', 'Ever had sexual intercourse',
       'Bought/sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent/long term infection', 'CD4 category', 'On ART',
       'ART initiated past 12 months those ever on ART',
       'Whether ARVs detected']    
     
F_X_train_ready_selected = F_X_train_ready[F_selected_features]
M_X_test_ready_selected = M_X_test_ready[M_selected_features]
F_X_test_ready_selected = F_X_test_ready[F_selected_features]
CM_test_ready_selected = CM_test_ready[M_selected_features]
CF_test_ready_selected = CF_test_ready[F_selected_features]
     
M_en_selected = load("en_sample\M_en_sample_data.joblib")
F_en_selected = load("en_sample\F_en_sample_data.joblib")
CM_en_selected = load("en_sample\M_en_sample_data.joblib")
CF_en_selected = load("en_sample\F_en_sample_data.joblib")

M_best_index = M_en_selected.best_index_
F_best_index = F_en_selected.best_index_

M_Y_train_selected_pred = M_en_selected.predict(M_X_train_ready_selected)
# M_Y_train_selected_pred_thresh = (M_xgb_selected.predict_proba(M_X_train_selected)[:,1] >= 0.3).astype(bool) # set threshold as 0.3
F_Y_train_selected_pred = F_en_selected.predict(F_X_train_ready_selected)

print('Confusion matrix on the  M train dataset:', confusion_matrix(M_Y_train_ready, M_Y_train_selected_pred))
print('Average F1 score on the  M train sample', "{:.1%}".format(M_en_selected.cv_results_['mean_test_f1'][M_best_index]))
print('CI F1 score on the M train sample', "{:.1%}".format(2*M_en_selected.cv_results_['std_test_f1'][M_best_index]))
print('Average Sensitivity on the M train sample', "{:.1%}".format(M_en_selected.cv_results_['mean_test_recall'][M_best_index]))
print('CI Sensitivity on the M train sample', "{:.1%}".format(2*M_en_selected.cv_results_['std_test_recall'][M_best_index]))
print('Average Positive Predictive Value on the M train sample', "{:.1%}".format(M_en_selected.cv_results_['mean_test_precision'][M_best_index]))
print('CI Positive Predictive Value on the M train sample', "{:.1%}".format(2*M_en_selected.cv_results_['std_test_precision'][M_best_index]))
print('Parameters', M_en_selected.best_estimator_)


print('Confusion matrix on the F train dataset:', confusion_matrix(F_Y_train_ready, F_Y_train_selected_pred))
print('Average F1 score on the F train sample', "{:.1%}".format(F_en_selected.cv_results_['mean_test_f1'][F_best_index]))
print('CI F1 score on the F train sample', "{:.1%}".format(2*F_en_selected.cv_results_['std_test_f1'][F_best_index]))
print('Average Sensitivity on the F train sample', "{:.1%}".format(F_en_selected.cv_results_['mean_test_recall'][F_best_index]))
print('CI Sensitivity on the F train sample', "{:.1%}".format(2*F_en_selected.cv_results_['std_test_recall'][F_best_index]))
print('Average Positive Predictive Value on the F train sample', "{:.1%}".format(F_en_selected.cv_results_['mean_test_precision'][F_best_index]))
print('CI Positive Predictive Value on the F train sample', "{:.1%}".format(2*F_en_selected.cv_results_['std_test_precision'][F_best_index]))
print('Parameters', F_en_selected.best_estimator_)

In [ ]:
from matplotlib import pyplot   
M_Y_test_selected_pred = M_en_selected.predict(M_X_test_ready_selected)
F_Y_test_selected_pred = F_en_selected.predict(F_X_test_ready_selected)
CM_Y_test_selected_pred = M_en_selected.predict(CM_test_ready_selected)
CF_Y_test_selected_pred = F_en_selected.predict(CF_test_ready_selected)



print('Confusion matrix on the M test dataset:', confusion_matrix(M_Y_test_ready, M_Y_test_selected_pred))
print('F1 score on the M test dataset:', "{:.1%}".format(f1_score(M_Y_test_ready, M_Y_test_selected_pred)))

print('Sensitivity on the M test dataset:', "{:.1%}".format(recall_score(M_Y_test_ready, M_Y_test_selected_pred)))
print('Positive Predictive Value on the M test dataset:', "{:.1%}".format(precision_score(M_Y_test_ready, M_Y_test_selected_pred)))


print('Confusion matrix on the F test dataset:', confusion_matrix(F_Y_test_ready, F_Y_test_selected_pred))
print('F1 score on the F test dataset:', "{:.1%}".format(f1_score(F_Y_test_ready, F_Y_test_selected_pred)))
print('Sensitivity on the F test dataset:', "{:.1%}".format(recall_score(F_Y_test_ready, F_Y_test_selected_pred)))
print('Positive Predictive Value on the F test dataset:', "{:.1%}".format(precision_score(F_Y_test_ready, F_Y_test_selected_pred)))

print('Confusion matrix on the CF  test dataset:', confusion_matrix(CF_Y_test_ready, CF_Y_test_selected_pred))
print('F1 score on the CF test dataset:', "{:.1%}".format(f1_score(CF_Y_test_ready, CF_Y_test_selected_pred)))
print('Sensitivity on the CF test dataset:', "{:.1%}".format(recall_score(CF_Y_test_ready, CF_Y_test_selected_pred)))
print('Positive Predictive Value on the CF test dataset:', "{:.1%}".format(precision_score(CF_Y_test_ready, CF_Y_test_selected_pred)))

print('Confusion matrix on the CM test dataset:', confusion_matrix(CM_Y_test_ready, CM_Y_test_selected_pred))
print('F1 score on the CM test dataset:', "{:.1%}".format(f1_score(CM_Y_test_ready, CM_Y_test_selected_pred)))
print('Sensitivity on the CM test dataset:', "{:.1%}".format(recall_score(CM_Y_test_ready, CM_Y_test_selected_pred)))
print('Positive Predictive Value on the CM test dataset:', "{:.1%}".format(precision_score(CM_Y_test_ready, CM_Y_test_selected_pred)))



def pr_f1_curve(model, dx_test, dy_test):
         dy_test_predict_proba = model.predict_proba(dx_test)[:,1]
         dy_test_predict = model.predict(dx_test)
         ppv_curve, sensitivity_curve, _ = precision_recall_curve(dy_test, dy_test_predict_proba)
         F1_score, AUC_score = f1_score(dy_test, dy_test_predict), auc(sensitivity_curve, ppv_curve)
         print(': F1 score = %.2f - Area under Curve = %.2f' % (F1_score, AUC_score))
         prevalence = np.sum(dy_test)/len(dy_test)
         pyplot.plot([0, 1], [prevalence, prevalence], linestyle='--', label='Random Testing')
         pyplot.plot(sensitivity_curve, ppv_curve, marker='.', label='Algorithmic Testing')
         pyplot.plot(sensitivity_curve, 2*(sensitivity_curve*ppv_curve)/(sensitivity_curve+ppv_curve), marker='', label='F1')
         pyplot.xlabel('Sensitivity')
         pyplot.ylabel('Positive Predicted Value')
         pyplot.legend(bbox_to_anchor=(0.45, 0.75))
         pyplot.show()
        
    
        
pr_f1_curve(M_en_selected, M_X_test_ready_selected, M_Y_test_ready)
pr_f1_curve(F_en_selected, F_X_test_ready_selected, F_Y_test_ready)
pr_f1_curve(CF_en_selected, CF_test_ready_selected, CF_Y_test_ready)
pr_f1_curve(CM_en_selected, CM_test_ready_selected, CM_Y_test_ready)


# Explain model predictions using shapley value
M_explainer = shap.TreeExplainer(M_en_selected.best_estimator_)
M_shap_values = M_explainer.shap_values(M_X_train_ready_selected)
F_explainer = shap.TreeExplainer(F_en_selected.best_estimator_)
F_shap_values = F_explainer.shap_values(F_X_train_ready_selected)

CF_explainer = shap.TreeExplainer(CF_en_selected.best_estimator_)
CF_shap_values = CF_explainer.shap_values(CF_test_ready_selected)

CM_explainer = shap.TreeExplainer(CM_en_selected.best_estimator_)
CM_shap_values = CM_explainer.shap_values(CM_test_ready_selected)

# Plot summary_plot
shap.summary_plot(M_shap_values, M_X_train_ready_selected, max_display=24)
shap.summary_plot(F_shap_values, F_X_train_ready_selected, max_display=29)

shap.summary_plot(CM_shap_values, CM_test_ready_selected, max_display=24)